In [1]:
import os, time
import numpy as np
import pandas as pd
import openpyxl

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# LDA (Phase 2 only)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 5 classifiers
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


In [ ]:
# -------------------------
# Paths
# -------------------------
DATA_ROOT = "Datasets"
RESULTS_XLSX = "/Users/classroomservices/Desktop/Winter/Machine Learning/Project/Code/Datasets/Results.xlsx"   # your template file
LABEL_COL = "Label"
RANDOM_STATE = 42

# -------------------------
# CV (Outer must be 10)
# -------------------------
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

# Inner tuning CV: not specified by prof -> 3 folds speeds up a lot
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

# -------------------------
# Parallelism (IMPORTANT)
# -------------------------
# If your run "stucks", set N_JOBS = 1 (most stable).
# On Mac, too much parallelism can stall in nested CV.
N_JOBS = 2   # try 2 first; if stuck -> set to 1

# -------------------------
# Random search iterations
# -------------------------
# More iterations = better tuning but slower.
# Typical balanced: 15–25.
N_ITER = 20
